In [2]:
!nvidia-smi5

/bin/bash: line 1: nvidia-smi5: command not found


In [3]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
  print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
else:
  print("No GPU")


No GPU


In [4]:
!pip install requests==2.31.0

In [5]:
!pip install datasets transformers[sentencepiece] requests==2.31.0 sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [6]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [8]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [42]:
raw_datasets['test'][1]

{'translation': {'en': "As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.",
  'hi': 'जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।'}}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
tokenizer("hello,how are you buddy ?")

{'input_ids': [39915, 2, 12561, 54, 27, 14445, 44, 22, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer("hello,how are you buddy ?","Happy coding")

{'input_ids': [39915, 2, 12561, 54, 27, 14445, 44, 22, 9295, 7308, 232, 137, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer(["hello,how are you buddy ?","Happy coding"])

{'input_ids': [[39915, 2, 12561, 54, 27, 14445, 44, 22, 0], [9295, 7308, 232, 137, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [15]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["इसका समाधान छोटे से ब्लैक बॉक्स में देख"]))

{'input_ids': [[738, 9122, 1775, 12, 20501, 8323, 11, 676, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
max_input_length =128
max_target_length = 128
source_lang = "en"
target_lang = "hi"
def preprocess_function(examples):
  inputs = [ex[source_lang] for ex in examples['translation']]
  targets = [ex[target_lang] for ex in examples['translation']]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation = True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation = True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


In [17]:
preprocess_function(raw_datasets['train'][:34])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0], [81, 1724, 1907, 6461, 34, 4, 5571, 10061, 0], [81, 1724, 1907, 6461, 34, 4, 3346, 10061, 0], [238, 583, 8, 4442, 26, 54, 6804, 97, 1724, 0], [6941, 9583, 0], [81, 9583, 8, 4, 8314, 3613, 121, 12321, 8165, 22403, 0], [6941, 5438, 3111, 0], [81, 3111, 10, 21900, 8, 4, 8314, 5438, 3, 0], [6941, 2740, 3111, 0], [81, 3111, 10, 21900, 8, 4, 8314, 2740, 3, 0], [10642, 9405, 0], [8945, 4, 2829, 7562, 8, 4, 899, 8165, 0], [4481, 4207, 13587, 0], [14926, 0], [16593, 0], [4595, 0], [56, 48023, 27350, 0], [38341, 17335, 34, 40471, 3757, 597, 8165, 0], [6207, 17604, 0], [106, 14092, 14005, 0], [643, 106, 44, 25772, 5840, 0], [11168, 0], [3782, 2501, 0], [3782, 8165, 0], [3000, 0], [6207, 14092, 0], [515, 16, 2173, 79, 48, 5314, 72, 597, 6927, 10, 8093, 0], [6941, 972, 2022, 4683, 0], [1972, 351, 2907, 2022, 16778, 0], [3171, 2022, 4865, 0], [17, 2991, 4222, 13, 0], [4417, 0], [515, 0]], 'attention_mask': [[1, 1, 1, 1,

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [19]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [20]:
batch_size = 16
learning_rate =2e-5
weight_decay=0.01
num_train_epochs = 1

In [24]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model,return_tensors="np")

In [25]:
import numpy as np

In [27]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size = batch_size,
    shuffle = True,
    collate_fn = data_collator,
)

In [28]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [41]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator,
)

In [35]:
optimizer=AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [36]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=num_train_epochs)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
156/156 [==============================] - 2510s 15s/step - loss: 3.7568 - val_loss: 3.9535


In [37]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


In [38]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [39]:
input_text  = "Hello brother.How are you ?"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[61949 16162   443     2    73   280    51    22     0]], shape=(1, 9), dtype=int32)


In [40]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

हैलो भाई, तुम कैसे हो?


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [64]:
def generate_translation(input_text):
  tokenized = tokenizer([input_text], return_tensors='np')
  out = model.generate(**tokenized, max_length=128)
  with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

In [65]:
generate_translation("question paper is tricky.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


'प्रश्न पत्र मुश्किल है।'

In [74]:
generate_translation("Men and women are enjoying party.")

'पुरुष और महिला पार्टी का आनंद ले रहे हैं।'

In [75]:
generate_translation("car is moving at a high speed.")

'कार उच्च गति से आगे बढ़ रही है।'

In [78]:
generate_translation("India is first country to reach on south moon.")

'भारत दक्षिण चाँद पर पहुंचने के लिए पहला देश है।'

In [79]:
generate_translation("I am proud to be Indian.")

'मुझे भारतीय होने पर गर्व है।'

In [81]:
generate_translation("Today World is moving towards acheiving great heights in the domain of technology.")

'आज दुनिया को प्रौद्योगिकी के क्षेत्र में विशाल ऊँचाई की ओर बढ़ रहा है।'